# Sorting Networks

## Daniel, Swati, and Michael

### Classes for Sorting Networks
Elements are objects that take in two inputs, A and B, and then output L and H with L = A, H = B if A < B or L = B, H = A if A > B. Each element instance tracks whether the two inputs were swapped. 

Layers are arrays of elements

In [3]:
import random

class Element:
    
    def swap(self, A, B, descending):   
        if A <= B:
            self.swapped = False
            self.L = A
            self.H = B
        else:
            self.swapped = True
            self.L = B
            self.H = A
        
        if descending:
            temp = self.L
            self.L = self.H
            self.H = temp
            self.swapped = not self.swapped
            
    def __init__(self, A, B, descending = False):
        self.prob = 1
        self.A = A
        self.B = B
        self.descending = descending
        self.swap(A, B, descending)
        
class Layer:
    def __init__(self, elements, indices):
        self.in_indices = indices
        self.elements = elements
        self.next = None
        self.out_indices = []
        for i, element in enumerate(elements):
            if element.swapped:
                self.out_indices.append(self.in_indices[i * 2 + 1])
                self.out_indices.append(self.in_indices[i * 2])
            else:
                self.out_indices.append(self.in_indices[i * 2])
                self.out_indices.append(self.in_indices[i * 2 + 1])
    
    def get_numbers(self):
        numbers = []
        for element in self.elements:
            numbers.append(element.L)
            numbers.append(element.H)
        return numbers
    
    def get_inputs(self):
        numbers = []
        for element in self.elements:
            numbers.append(element.A)
            numbers.append(element.B)
        return numbers
    
    def print_layer(self):
        print(self.get_numbers())
        
    def get_end_layer(self):
        current_layer = self
        while current_layer.next is not None:
            current_layer = current_layer.next
        return current_layer
    
    def get_end(self):
        return self.get_end_layer().get_numbers()
    
    def print_end(self):
        print(self.get_end())
    
    def get_depth(self):
        depth = 1
        current_layer = self
        while current_layer.next is not None:
            current_layer = current_layer.next
            depth = depth + 1
        return depth
    
    def copy(self, layer):
        self.elements = layer.elements
        self.next = layer.next
        self.in_indices = layer.in_indices
        self.out_indices = layer.out_indices
        
    def count_swaps(self):
        swaps = 0
        current_layer = self
        while current_layer is not None:
            for element in current_layer.elements:
                if element.swapped:
                    swaps = swaps + 1
            current_layer = current_layer.next
        return swaps
    
    def to_list(self):
        layer_list = []
        current_layer = self
        while current_layer is not None:
            layer_list.append(current_layer.elements)
            current_layer = current_layer.next
        return layer_list

def combine_sublayers(sublayers):
    if len(sublayers) is 0:
        return None
    else:
        total_elements = []
        next_sublayers = []
        indices = []
        for sublayer in sublayers:
            if sublayer.next is not None:
                next_sublayers.append(sublayer.next)
            total_elements = total_elements + sublayer.elements
            indices = indices + sublayer.in_indices
        current_layer = Layer(total_elements, indices)
        current_layer.next = combine_sublayers(next_sublayers)

        return current_layer


In [20]:
def bitonic_sorter(a, indices, reverse = False):
    # print("SORTING {} WITH INDICES {}".format(a, indices))
    if len(a) == 2:
        # If there are only two elements to sort, the layer is only one comparator
        layer = Layer([Element(a[0], a[1], reverse)], indices)
        return layer
    else:
        # print("CALLING BITONIC SORTER")
        # Else, we create layer with inputs (1, n/2 + 1), (2, n/2 + 2), ...
        bitonic_halver = []
        halver_indices = []
        for i in range(0, len(a) // 2):
            bitonic_halver.append(Element(a[i], a[i + len(a) // 2], reverse))
            halver_indices.append(indices[i])
            halver_indices.append(indices[i + len(a) // 2])
        layer = Layer(bitonic_halver, halver_indices)
        low_numbers = []
        high_numbers = []
        out_indices = layer.out_indices
        # All low elements of the last layer get passed into another bitonic sorter
        # All high elements of the last layer get passed into another bitonic sorter 
        low_indices = []
        high_indices = []
        for i in range(0, len(layer.elements)):
            low_numbers.append(layer.elements[i].L)
            high_numbers.append(layer.elements[i].H)
            low_indices.append(out_indices[2 * i])
            high_indices.append(out_indices[2 * i + 1])

        # print("Low indices: {}".format(low_indices))
        # print("High indices: {}".format(high_indices))
        # print()
        lower_layer = bitonic_sorter(low_numbers, low_indices, reverse)
        higher_layer = bitonic_sorter(high_numbers, high_indices, reverse)
        
        # We combine the outputs of the low layer
        layer.next = combine_sublayers([lower_layer, higher_layer])
        indices = layer.out_indices
        return layer
    
    
def bitonic_converter(a):
    sort_size = 2
    top_layer = Layer([], [])
    current_layer = top_layer
    current_indices = list(range(0, len(a)))
    while sort_size < len(a):
        # Sort size is 2, 4, 8, ... 2^(n - 1)
        current_layer.next = Layer([], current_indices)
        current_layer = current_layer.next
        sublayers = []
        sublayer_out_indices = []
        reverse = False
        # Sort every sort_size elements in alternating up down sequences
        for i in range(0, len(a) // sort_size):
            sublayer_indices = current_indices[(i) * sort_size:(i + 1) * sort_size]
            sublayers.append(bitonic_sorter(a[(i) * sort_size:(i + 1) * sort_size], sublayer_indices, reverse))
            sublayer_out_indices = sublayer_out_indices + sublayers[i].get_end_layer().out_indices
            reverse = not reverse
        # Combine the sublayers and pass output into the next sorting layer
        current_layer.copy(combine_sublayers(sublayers))
        current_layer = current_layer.get_end_layer()
        current_indices = sublayer_out_indices
        # current_layer.out_indices = sublayer_out_indices
        a = current_layer.get_numbers()
        sort_size = sort_size * 2
    return top_layer.next

def sort(a):
    bitonic_layers = bitonic_converter(a)
    b = bitonic_layers.get_end()
    bitonic_indices = bitonic_layers.get_end_layer().out_indices
    sorting_layers = bitonic_sorter(b, bitonic_indices)
    bitonic_layers.get_end_layer().next = sorting_layers
    return bitonic_layers

In [ ]:
def perturb(vector, epsilon, rounding):
    perturbed = []
    for v in vector:
        r = round(random.uniform(-eps, eps), rounding)
        perturbed.append(v + r)
    return perturbed

## Mergesorting Perturbed Arrays

In [100]:
import numpy as np

random.seed(1)

P = 4    # arrays of size 2**p
S = 2**P # size of array
EPS = 1  # bound on perturbation
R = 2    # rounding of perturbation
v = list(range(0, S))
random.shuffle(v)
v_p = perturb(v, EPS, R)
v_all = np.array([v, v_p]).transpose()
print("ORIGINAL ARRAYS")
print(v_all)
print()

sorting_network = sort(v)
v_sorted = sorting_network.get_end()
sorted_mapping = sorting_network.get_end_layer().out_indices
v_p_sorted = [round(v_p[i], r) for i in sorted_mapping]

v_all_sorted = np.array([v_sorted, v_p_sorted]).transpose()
print("SORTED ARRAYS")
print(v_all_sorted)
print()

print("ARRAY TO MERGE")
print(np.transpose(np.array([v_p_sorted])))
print()
for p in range(0, P):
    print("-----STAGE {}-----".format(p + 1))
    sort_size = 2**(p + 1)

    for i in range(0, round(S / (sort_size))):
        mid_top = round(((i + 1/2) * sort_size))
        mid_bottom = mid_top - 1
        bottom_array = v_p_sorted[round(i * sort_size) : mid_top]
        top_array = v_p_sorted[mid_top: round((i + 1) * sort_size)]
        print("MERGING")
        print(np.transpose(np.array([bottom_array, top_array])))
        print()
        if (v_p_sorted[mid_bottom] > v_p_sorted[mid_top]):
            print("SWAPPING")
            print()
            temp = v_p_sorted[mid_bottom]
            v_p_sorted[mid_bottom] = v_p_sorted[mid_top]
            v_p_sorted[mid_top] = temp
print(v_p_sorted)

ORIGINAL ARRAYS
[[ 2.    1.06]
 [10.   10.67]
 [ 0.   -0.13]
 [14.   14.52]
 [ 6.    5.  ]
 [ 5.    4.89]
 [ 3.    3.44]
 [ 8.    7.46]
 [ 7.    7.89]
 [11.   11.8 ]
 [15.   14.06]
 [ 1.    0.05]
 [12.   12.08]
 [13.   13.88]
 [ 9.    8.76]
 [ 4.    3.43]]

SORTED ARRAYS
[[ 0.   -0.13]
 [ 1.    0.05]
 [ 2.    1.06]
 [ 3.    3.44]
 [ 4.    3.43]
 [ 5.    4.89]
 [ 6.    5.  ]
 [ 7.    7.89]
 [ 8.    7.46]
 [ 9.    8.76]
 [10.   10.67]
 [11.   11.8 ]
 [12.   12.08]
 [13.   13.88]
 [14.   14.52]
 [15.   14.06]]

ARRAY TO MERGE
[[-0.13]
 [ 0.05]
 [ 1.06]
 [ 3.44]
 [ 3.43]
 [ 4.89]
 [ 5.  ]
 [ 7.89]
 [ 7.46]
 [ 8.76]
 [10.67]
 [11.8 ]
 [12.08]
 [13.88]
 [14.52]
 [14.06]]

-----STAGE 1-----
MERGING
[[-0.13  0.05]]

MERGING
[[1.06 3.44]]

MERGING
[[3.43 4.89]]

MERGING
[[5.   7.89]]

MERGING
[[7.46 8.76]]

MERGING
[[10.67 11.8 ]]

MERGING
[[12.08 13.88]]

MERGING
[[14.52 14.06]]

SWAPPING

-----STAGE 2-----
MERGING
[[-0.13  1.06]
 [ 0.05  3.44]]

MERGING
[[3.43 5.  ]
 [4.89 7.89]]

MERGING
[[ 

[-0.13, 0.05, 1.06, 3.44, 3.43, 4.89, 5.0, 7.89, 7.46, 8.76, 10.67, 11.8, 12.08, 13.88, 14.52, 14.06]
-----STAGE 1-----

-----STAGE 2-----

-----STAGE 3-----

-----STAGE 4-----

[-0.13, 0.05, 1.06, 3.43, 3.44, 4.89, 5.0, 7.46, 7.89, 8.76, 10.67, 11.8, 12.08, 13.88, 14.06, 14.52]
